#### Intcode setup

- I have a really ugly method for handling an unknown memory space....just appending values 0-HIGH VAL. This was stated to do in the actual prompt, and I had missed it for quite some time. 

In [1]:

# empty list
data_list = []

# need to ensure a clean run each time 
with open('day9_data.txt') as f:
    for line in f:
        data_list.append(line.split(','))

# convert to ints
clean_d_list = [int(x) for x in data_list[0]]

In [2]:
extra_list = [x for x in range(1000000)]

In [3]:
day_9_tweak = clean_d_list + extra_list

In [4]:
def mode_check_1(mode, list_vals, init_val, rel_base):
    if mode == 0:
        value =  list_vals[init_val + 1]

    if mode == 1:
        value =  init_val + 1

    if mode == 2:
        value =  rel_base + list_vals[init_val + 1]
    
    return value

In [5]:
def mode_check_2(mode, list_vals, init_val, rel_base):
    if mode == 0:
        value = list_vals[init_val + 2]

    if mode == 1:
        value = init_val + 2

    if mode == 2:
        value = rel_base + list_vals[init_val + 2]
        
    return value

In [6]:
# This should handle 203 error
# Will allow for the handling of mode 2
def val_literal(mode, list_vals, init_val, rel_base):
    if mode == 0:
        value = list_vals[init_val + 3]

    if mode == 1:
        raise Exception("not allowed")

    if mode == 2:
        value = rel_base + list_vals[init_val + 3]
        
    return value    

In [7]:
def intcode_pc(list_vals, init_val, rel_base):
    
    """Built in Day 7"""
    
    while True:

        # identify the opt_code_str
        opt_code_str = str(list_vals[init_val])

        # take actual opt code: 
        opt_code = int(opt_code_str[-2:])

        # modes can be 0 == position or 1 == immediate 
        # handle mode with try / except
        mode_3, mode_2, mode_1 = 0, 0, 0

        # overwrite 0s UNLESS type error, then we know we had trailing 0s
        # saw example here: https://p.szy.io/MrmgEA/python
        try:
            mode_1 = int(opt_code_str[-3])
            mode_2 = int(opt_code_str[-4])
            mode_3 = int(opt_code_str[-5])

        except IndexError:
            pass


        # handling each type of opt code
        if opt_code == 1:

            # the third param provides the writing point: we always look at the 3rd value after opt code to write
            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)

            # get sum vals
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)
            ans = list_vals[val1] + list_vals[val2]

            # overwrite
            list_vals[write_ix] = ans

            init_val += 4

        elif opt_code == 2:

            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)

            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)
            ans = list_vals[val1] * list_vals[val2]

            # overwrite
            list_vals[write_ix] = ans
      
            init_val += 4

        elif opt_code == 3:

            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)
            list_vals[write_ix] = int(input())
            init_val += 2


        elif opt_code == 4:
            # output val that is at 'init_val + 1' index: think i need t be able to handle a non-0 val....
            # possibility of mode == 1?

            # something off with mode not being picked up properly
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            print(f"Val in 4 is {list_vals[val1]}")
   
            init_val += 2        

        elif opt_code == 5:

            # jump-if-true
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            if list_vals[val1] != 0:
                init_val = list_vals[val2]
            else:
                init_val += 3

        elif opt_code == 6:

            # jump-if-false
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            if list_vals[val1] == 0:
                init_val = list_vals[val2]
            else:
                init_val += 3

        elif opt_code == 7:

            # less than
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)

            if list_vals[val1] < list_vals[val2]:
                list_vals[write_ix] = 1
            else:
                list_vals[write_ix] = 0
                
            init_val += 4

        elif opt_code == 8:

            # equals
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            val2 = mode_check_2(mode_2, list_vals, init_val, rel_base)

            write_ix = val_literal(mode_3, list_vals, init_val, rel_base)

            if list_vals[val1] == list_vals[val2]:
                list_vals[write_ix] = 1
            else:
                list_vals[write_ix] = 0
                
            init_val += 4
            
        elif opt_code == 9:

            # update relative base
            val1 = mode_check_1(mode_1, list_vals, init_val, rel_base)
            rel_base += list_vals[val1]
            init_val += 2
            
        elif opt_code == 99:
            break

        else:

            print("Something is broken!")

In [8]:
intcode_pc(list_vals = day_9_tweak, init_val = 0, rel_base=0)

2
Val in 4 is 59785
